In [ ]:
import xarray as xr
import numpy as np
from utils.files import OIFSPreprocessor, NEMOPreprocessor
import convergence_checker as cc
import control_experiment_1 as ce1
import user_context as context

In [ ]:
max_iters = ce1.max_iters
start_date = ce1.start_date
exp_id = "C1SP"

In [ ]:
found_output_dirs = list(context.output_dir.glob(f"{exp_id}_*"))
if found_output_dirs == []:
    ce1.run_parallel_schwarz_without_cleanup()
    schwarz_dir_ref = (context.output_dir / exp_id).rename(
        context.output_dir / f"{exp_id}_{max_iters + 1}"
    )

In [ ]:
conv_checker = cc.ConvergenceChecker()
locals_final = []
amplitudes_final = []
for iter in range(2, max_iters + 1):
    schwarz_dir_iter = context.output_dir / f"{exp_id}_{iter}"
    local, amplitude = conv_checker.check_convergence(schwarz_dir_iter, schwarz_dir_ref)
    locals_final.append(local)
    amplitudes_final.append(amplitude)
    print(f"Iter {iter - 1}: {local=}, {amplitude=}")

In [ ]:
conv_checker = cc.ConvergenceChecker()
locals_subs = []
amplitudes_subs = []
for iter in range(2, max_iters + 1):
    schwarz_dir_iter = context.output_dir / f"{exp_id}_{iter}"
    schwarz_dir_next_iter = context.output_dir / f"{exp_id}_{iter + 1}"
    local, amplitude = conv_checker.check_convergence(
        schwarz_dir_iter, schwarz_dir_next_iter
    )
    locals_subs.append(local)
    amplitudes_subs.append(amplitude)
    print(f"Iter {iter - 1}: {local=}, {amplitude=}")

In [ ]:
nemo_preproc = NEMOPreprocessor(start_date)

penultimate_schwarz_dir = context.output_dir / f"{exp_id}_{max_iters - 1}"
final_schwarz_dir = context.output_dir / f"{exp_id}_{max_iters}"
nemo_penultimate_file = next(penultimate_schwarz_dir.glob("*_grid_T.nc"))
nemo_final_file = next(final_schwarz_dir.glob("*_grid_T.nc"))

penultimate_output = xr.open_mfdataset(
    nemo_penultimate_file, preprocess=nemo_preproc.preprocess
)
final_output = xr.open_mfdataset(nemo_final_file, preprocess=nemo_preproc.preprocess)

np.max(np.abs(final_output.sosstsst - penultimate_output.sosstsst)).load()

In [ ]:
oifs_preproc = OIFSPreprocessor(start_date)

oifs_penultimate_file = penultimate_schwarz_dir / "progvar.nc"
oifs_final_file = final_schwarz_dir / "progvar.nc"

penultimate_output = xr.open_mfdataset(
    oifs_penultimate_file, preprocess=oifs_preproc.preprocess
)
final_output = xr.open_mfdataset(oifs_final_file, preprocess=oifs_preproc.preprocess)

np.max(np.abs(final_output.t - penultimate_output.t)).load()